#### Install Libraries


In [ ]:
! pip install SQLAlchemy psycopg2 sqlalchemy-timescaledb

#### Import Libraries


In [1]:
import requests
import pandas as pd
from datetime import datetime, timezone
import psycopg2
import time
from dotenv import load_dotenv
import os

load_dotenv()
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

#### Configuration


In [2]:
CITY_IDS = [
    "1172451",
    "1174872",
    "1162015",
    "1174042",
    "1169684",
    "1176734",
    "1177662",
    "1177446",
    "1185056",
    "1184196",
]
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

#### Fetch/Process Weather Data


In [3]:
# Fetch real-time weather data for a single city
def fetch_current_weather(city_id):
    params = {"id": city_id, "appid": OPENWEATHER_API_KEY, "units": "metric"}
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

In [ ]:
weather_data = fetch_current_weather(1184196)
weather_data

{'coord': {'lon': 73.9104, 'lat': 33.9472},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 3.96,
  'feels_like': 0.67,
  'temp_min': 3.96,
  'temp_max': 3.96,
  'pressure': 1019,
  'humidity': 30,
  'sea_level': 1019,
  'grnd_level': 811},
 'visibility': 10000,
 'wind': {'speed': 3.86, 'deg': 47, 'gust': 4.65},
 'clouds': {'all': 0},
 'dt': 1732677217,
 'sys': {'country': 'PK', 'sunrise': 1732672093, 'sunset': 1732708574},
 'timezone': 18000,
 'id': 1184196,
 'name': 'Azad Kashmir',
 'cod': 200}

In [4]:
# Process and extract the required fields from the API response
def process_weather_data(weather_data):
    return {
        "city_id": weather_data["id"],
        "city_name": weather_data["name"],
        "forecast_time": datetime.fromtimestamp(weather_data["dt"], timezone.utc),
        "condition_id": weather_data["weather"][0]["id"],
        "condition_main": weather_data["weather"][0]["main"],
        "condition_description": weather_data["weather"][0]["description"],
        "condition_icon": weather_data["weather"][0]["icon"],
        "temperature": weather_data["main"]["temp"],
        "feels_like": weather_data["main"]["feels_like"],
        "humidity": weather_data["main"]["humidity"],
        "pressure": weather_data["main"]["pressure"],
        "cloudiness": weather_data["clouds"]["all"],
        "visibility": weather_data.get("visibility", None),
        "wind_speed": weather_data["wind"]["speed"],
        "wind_degrees": weather_data["wind"].get("deg", None),
        "rain": weather_data.get("rain", {}).get("1h", 0.0),  # Rain in the last 1 hour
        "snow": weather_data.get("snow", {}).get("1h", 0.0),  # Snow in the last 1 hour
        "sunrise": datetime.fromtimestamp(weather_data["sys"]["sunrise"], timezone.utc),
        "sunset": datetime.fromtimestamp(weather_data["sys"]["sunset"], timezone.utc),
    }

In [ ]:
while True:
    weather_records = []

    for city_id in CITY_IDS:
        print(f"Fetching data for city ID: {city_id}...")
        weather_data = fetch_current_weather(city_id)
        processed_data = process_weather_data(
            weather_data
        )  # Process and extract fields
        weather_records.append(processed_data)

    df = pd.DataFrame(weather_records)
    print("Data fetching completed!")
    print(df.head())

    # Save DataFrame to a CSV file (appending data)
    df.to_csv("current_weather_details.csv", mode="a", header=False, index=False)
    print("Data saved to current_weather_details.csv")

    # Wait for 1 minute (60 seconds) before the next fetch
    time.sleep(60)

Fetching data for city ID: 1172451...
Fetching data for city ID: 1174872...
Fetching data for city ID: 1162015...
Fetching data for city ID: 1174042...
Fetching data for city ID: 1169684...
Fetching data for city ID: 1176734...
Fetching data for city ID: 1177662...
Fetching data for city ID: 1177446...
Fetching data for city ID: 1185056...
Fetching data for city ID: 1184196...
Data fetching completed!
   city_id  city_name             forecast_time  condition_id condition_main  \
0  1172451     Lahore 2024-11-25 11:51:58+00:00           800          Clear   
1  1174872    Karachi 2024-11-25 11:50:54+00:00           721           Haze   
2  1162015  Islamabad 2024-11-25 11:54:12+00:00           800          Clear   
3  1174042    Kharian 2024-11-25 11:56:10+00:00           800          Clear   
4  1169684     Murree 2024-11-25 11:56:11+00:00           800          Clear   

  condition_description condition_icon  temperature  feels_like  humidity  \
0             clear sky            01

KeyboardInterrupt: 